# Starter notebook ”Dogs vs. Cats” with EfficientNet vs My own model
This notebook shows a flow of "Dogs and Cats" distinguish model development.  
https://www.kaggle.com/c/dogs-vs-cats/  

[Agenda]
1. Import libraries  
2.　Data preparation  
    2-1. Unzip data  
    2-2. Path and file name setting  
    2-3. making directory tree for ImageDataGenerator  
    2-4. Copy picture data to tree for ImageDataGenerator  
3. Training  
    3-1. setting for training  
    3-2. dataset confirmation  
    3-3. ImageDataGenerator  
    3-4. Network definition  
    3-5. train  
4. Save Model  
5. Model evaluation  
    5-1. Confusion matrix  
6. Prediction with Test data  
7. Create submission file  

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Check　assined GPU detail
!nvidia-smi

## 1. Import libraries

In [ ]:
# File I/O
import csv
import subprocess
import shutil
import os
from glob import glob
from datetime import datetime
from PIL import Image

# Data processing
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold

# Image processing
import cv2
from scipy.ndimage import rotate
import scipy.misc

# Graph
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import tensorflow as tf
from keras import Model
from keras import optimizers
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten, Dropout
from keras.layers.core import Dense
from keras.utils.np_utils import to_categorical
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.optimizers import Adagrad
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping
# EfficientNet
from tensorflow.keras.applications import EfficientNetB0

from tensorflow.keras.optimizers import RMSprop

from IPython.display import display
import ipywidgets as widgets

# 2.Data preparation  
## 2-1. Unzip data  
I will use ImageGenerator to load data for training.  
Test data poctures shall not be stored in Test data path folder directory.Test data path shall have one folder.  
If prediction target pictures are stored in the test data path folder directory, error occurrs.   

In [ ]:
!unzip -q '../input/dogs-vs-cats/train.zip' -d '/kaggle/working/' # ->'/kaggle/working/train'
!unzip -q '../input/dogs-vs-cats/test1.zip' -d '/kaggle/working/test/' # ->'/kaggle/working/test/test1'

In [ ]:
shutil.move('/kaggle/working/train/', '/kaggle/working/train_raw/all_pics')

## 2-2. Path and file name setting  
Again, test path folder shall have one more folder at least.  

In [ ]:
root_path = '/kaggle/working'

# Path of Unzipping data stored (root)
train_raw_root_path = '/kaggle/working/train_raw/'


# Path of Unzipping data stored 
train_raw_path = '/kaggle/working/train_raw/all_pics'

# Path for training
train_root_path = '/kaggle/working/train_root'
test_path = '/kaggle/working/test'
submit_file = 'submission.csv'

# Path for Imagegenerator
train_dir = os.path.join(train_root_path, 'train')
valid_dir = os.path.join(train_root_path, 'valid')

print(train_dir)
print(valid_dir)

## 2-3. making directory tree for ImageDataGenerator

Making each file name list of classificaion target categories

## 2-4. Copy picture data to tree for ImageDataGenerator

In [ ]:
# picture file name list
data_img_list = [os.path.basename(f) for f in glob(f'{train_raw_path}/*.jpg')]
test_img_list = [os.path.basename(f) for f in glob(f'{test_path}/test1/*.jpg')]

In [ ]:
# target labels list
target_labels = ['cat', 'dog']

# directories in the train_root path
dirs = ['train', 'valid']

# make dirctories for data store
for d in dirs:
    for label in target_labels:
        os.makedirs(os.path.join(train_root_path, d, label), exist_ok=True)

In [ ]:
# Initializing instance
img_cat_list = []
img_dog_list = []

# make dog and cat file name list
for f in data_img_list:
    label = f.split('.')[0]
    if label == target_labels[0]:
        img_cat_list.append(f)
    elif label == target_labels[1]:
        img_dog_list.append(f)
    else:
        print('abnormal file name is found', f)

# Split data to for train and for valid
train_list_cat, val_list_cat, train_list_dog, val_list_dog = train_test_split(img_cat_list, img_dog_list, 
                                                                  test_size=0.3,
                                                                 random_state=46)
# directories list to join to train_root path
copy_to_dirs = ['train/cat', 'valid/cat', 'train/dog', 'valid/dog']
img_lists = [train_list_cat, val_list_cat, train_list_dog, val_list_dog] 

# Copy to each directories
for to_dir in copy_to_dirs:
    for f in img_lists[copy_to_dirs.index(to_dir)]:
        shutil.copyfile(os.path.join(train_raw_path,f), os.path.join(train_root_path, to_dir,f))

In [ ]:
# Check file quantity 
print('train image quantity:', len(data_img_list))
print('cat:', len(img_cat_list))
print('dog:', len(img_dog_list))
print()
print('prediction target test image quantity:', len(test_img_list))

In [ ]:
# Check file number
for d in dirs:
    for label in target_labels:
        print(os.path.join(train_root_path, d, label))
        print(len([file for file in os.listdir(os.path.join(train_root_path, d, label))]))

Check file quantity in each directory.Above quantity shows correct copy picture process.

In [ ]:
# Check data directry tree structure
!tree -d '/kaggle/working/'

test: pictures for test  
train_raw: train data provided from kaggle  
train_root: train pictures copied to place for ImageDataGenerator

# 3. Training

## 3-1. setting for training

In [ ]:
learning_rate = 1e-4
batch_size = 20
input_height , input_width = 300, 300
random_split = 1
epochs = 200

# model name setting
model_name_list = ['efficient_net', 'mymodel', ]

## 3-2. dataset confirmation

In [ ]:
# 画像の表示
im = Image.open(os.path.join(train_raw_path, img_dog_list[0]))
print(os.path.join(train_raw_path, img_dog_list[0]))
plt.imshow(im)
plt.title(img_dog_list[0])
plt.axis("off")
plt.show()

In [ ]:
labels = ['cat', 'dog']
label_description = {
    '0': 'cat',
    '1': 'dog',
}

for label in labels:
    f, ax = plt.subplots(figsize=(12,10))
    if label == 'cat':
        img_list = img_cat_list
        
    elif label == 'dog':
        img_list = img_dog_list
        
    # show pictures 3 x 3 
    for x in range(9):
        plt.subplot(3, 3, x+1)
        im = Image.open(os.path.join(train_raw_path, img_list[x]))
        plt.axis('off')
        plt.title(img_list[x])
        plt.imshow(im)
        
    print(f'\t\t\t\t# {label}')
    plt.show()
    print()

## 3-3. ImageDataGenerator

In [ ]:
# All images will be rescaled by 1./255
# train_datagen = ImageDataGenerator(rescale=1./255)
# Added data augumentation to avoid over fitting (version 8)
train_datagen = ImageDataGenerator(
    rotation_range = 360,
    # zca_whitening = True,
    # zca_epsilon = 1e-06,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.1,
    zoom_range = [0.5,1.0],
    channel_shift_range = 0.2,
    horizontal_flip = True,
    vertical_flip = True,
    rescale = 1./255,
    preprocessing_function = None
)
valid_datagen = ImageDataGenerator(rescale=1./255)


# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=(input_height, input_width),  # All images will be resized to 150x150
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow validation images in batches of 20 using test_datagen generator
valid_generator = valid_datagen.flow_from_directory(
        valid_dir,
        target_size=(input_height, input_width),
        batch_size=20,
        class_mode='binary')

## 3-4. Network definition

In [ ]:
def efficientnet_model(input_height, input_width):
    model = Sequential()
    model.add( tf.keras.applications.EfficientNetB3(
    include_top=False,
    weights="imagenet", input_shape=(input_height, input_width, 3)))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(1, activation="sigmoid"))
    
    return model

In [ ]:
def mymodel(input_height, input_width):
    # モデルの作成
    mymodel = Sequential()
    mymodel.add(Conv2D(32, kernel_size=3, padding="same", activation='relu', input_shape=(input_height, input_width, 3)))
    mymodel.add(MaxPooling2D(pool_size=(3, 3)))
    mymodel.add(Conv2D(64, kernel_size=3, padding="same", activation='relu'))
    mymodel.add(MaxPooling2D(pool_size=(2, 2)))
    mymodel.add(Conv2D(128, kernel_size=3, padding="same", activation='relu'))
    mymodel.add(MaxPooling2D(pool_size=(2, 2)))
    mymodel.add(Flatten())    #Flatten()により特徴マップをベクトルに変換し、後続の全結合層と繋げられるようにする
    mymodel.add(Dense(384, activation='relu'))
    mymodel.add(Dense(128, activation='relu'))
    mymodel.add(Dense(1, activation='sigmoid'))    #Softmax関数にて、クラス毎の確率として出力

    return mymodel

Adding some model network definition

In [ ]:
'''
def mymodel2 (input_height, input_width, num_classes):
  # network layer
  # write code and define network

  return model
'''

In [ ]:
def get_answer(x):
    return x

In [ ]:
model_selected = get_answer(widgets.RadioButtons(options=model_name_list))
display(model_selected)

In [ ]:
model_name = model_selected.value
print(model_name)

In [ ]:
def select_model(model_name, input_height, input_width):
    if model_name == 'mymodel':
        model = mymodel(input_height, input_width)
    elif model_name == 'efficient_net':
        model = efficientnet_model(input_height, input_width)

    return model

model = select_model(model_name, input_height, input_width)

# display model summary 
model.summary()

## 3-5. train

In [ ]:
# compile model with some setting item
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=learning_rate),
              metrics=['accuracy'])

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_accuracy',
                                           patiene = 3,
                                           verbose = 1,
                                           factor = 0.5,
                                           min_lr = 0.00001)

early_stopping = EarlyStopping(monitor='val_loss',
                               patience=10)

In [ ]:
# train
history = model.fit(
      train_generator,
      steps_per_epoch=100,  # 2000 images = batch_size * steps
      epochs=200,
      validation_data=valid_generator,
      validation_steps=50,  # 1000 images = batch_size * steps
      verbose=2,
      callbacks = [learning_rate_reduction, early_stopping],
)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

# 4. Save Model

In [ ]:
# Save Model
model.save(root_path + '/model.h5' )    # model
model.save_weights(root_path + '/model_weights.h5')    # model parameter

In [ ]:
# load model
# from keras.models import load_model
# model = load_model(root_path + '/model.h5')

# 5. Model evaluation

## 5-1. Confusion matrix

In [ ]:
eval_datagen = ImageDataGenerator(rescale=1./255)
eval_generator = eval_datagen.flow_from_directory(
    train_raw_root_path,
    target_size=(input_height, input_width),
    batch_size=1,
    class_mode=None,
    shuffle=False)

print('evaluation(all pictures(raw train dataset) ：', len(data_img_list))

In [ ]:
eval_pred_proba = model.predict(
    eval_generator,
    steps=len(data_img_list),
    verbose=0)

# probability value is predicted
print(eval_pred_proba.shape)
eval_pred_class = np.where(eval_pred_proba < 0.5, 0, 1)

In [ ]:
true_class = []
true_class_name = []
for f in os.listdir(train_raw_path):
    class_name = f.split('.')[0]
    true_class_name.append(class_name)
    if class_name == 'cat':
        true_class.append(0)
    elif class_name == 'dog':
        true_class.append(1)
    else:
        continue
        
print(true_class[0:10])
print(true_class_name[0:10])

In [ ]:
from sklearn.metrics import confusion_matrix

eval_cmx = confusion_matrix(true_class, eval_pred_class)

# DataFrame
eval_cmx_df = pd.DataFrame(eval_cmx, columns=target_labels, index=target_labels)

print('Confution matrix with train all pics data')

# 結果の表示
eval_cmx_df

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

print('accuracy:', accuracy_score(true_class, eval_pred_class))
print('precision:', precision_score(true_class, eval_pred_class, average='macro'))
print('recall:', recall_score(true_class, eval_pred_class, average='macro'))
print('f1:', f1_score(true_class, eval_pred_class, average='macro'))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.heatmap(eval_cmx, annot=True, cmap='Blues')
plt.savefig(root_path + '/' + 'sklearn_confusion_matrix.png')

## 6. Prediction with Test data

In [ ]:
# Again, test path folder shall have one more folder at least and test data shall be stored in the folder
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(input_height, input_width),
    batch_size=1,
    class_mode=None,
    shuffle=False)

print('test data quantity：', len(test_img_list))

In [ ]:
test_pred_proba = model.predict(
    test_generator,
    steps=len(test_img_list),
    verbose=0)

print(test_pred_proba.shape)
test_pred_class = np.where(test_pred_proba < 0.5, 0, 1)

## 7. Create Submission file

In [ ]:
test_filename_list=[]
for f in test_img_list:
    test_filename_list.append(f.split('.')[0])

In [ ]:
result_id = pd.DataFrame(test_filename_list, columns=['id'])
result_label = pd.DataFrame(test_pred_class, columns=['label'])
result_for_submit = pd.concat([result_id, result_label], axis=1)
result_for_submit.head()

In [ ]:
result_for_submit.to_csv(submit_file, sep=",", header=True, index=False)

------------------------unimplimented-------------------------------

In [ ]:
'''
# ファイル名に付ける日時データの文字列作成
import datetime

dt_now = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
d_today = datetime.date.today()

filetime = str(d_today)+'_'+str(dt_now.hour)+str(dt_now.minute)
'''

In [ ]:
'''
hist  = model.fit(
    train_generator,
    validation_data = valid_generator,
    steps_per_epoch = len(df_train)*0.9//batch_size,
    validation_steps = len(df_train)*0.1//batch_size, 
    epochs = epochs,
    # callbacks = [model_checkpoint, early_stopping])
    callbacks = [model_checkpoint, learning_rate_reduction, early_stopping])
'''

In [ ]:
'''
# cp_cb = ModelCheckpoint(model_path + '/checkpoint/' + "{epoch}"+"checkpoint.h5", verbose=1, save_best_only=True)
model_checkpoint = ModelCheckpoint(
        filepath = f'{root_path}/checkpoint.h5',
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True)
'''